In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1579143173633_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-3>

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
ss = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Preprocessing data loaded from S3

In [4]:
fname = 's3://msds697-g2/Wrist/S_2.csv'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
wrist2 = ss.read.csv(fname, header=True, inferSchema=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
def clean_df(df, sub_id):
    df = df.filter(df.TEMP.isNotNull())
    df = df.withColumn('sub_id', lit(f'S_{sub_id}'))
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
wrists = clean_df(wrist2, 2)
nums = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
for i in nums[1:]:
    wrist = ss.read.csv(f's3://msds697-g2/Wrist/S_{i}.csv', header=True, inferSchema=True)
    wrist = clean_df(wrist, i)
    wrists = wrists.union(wrist)
wrists.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[_c0: double, label: double, ACC_0: double, ACC_1: double, ACC_2: double, BVP: double, EDA: double, TEMP: double, sub_id: string]

In [8]:
wrists = wrists.filter(wrists.label.isin([1, 2, 3]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
wrists.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

132608

In [10]:
wrists.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-----+-----+-----+-----+------+--------+-----+------+
|   _c0|label|ACC_0|ACC_1|ACC_2|   BVP|     EDA| TEMP|sub_id|
+------+-----+-----+-----+-----+------+--------+-----+------+
|306.75|  1.0| 65.0| 12.0| 21.0|-33.58|1.640539|35.81|   S_2|
| 307.0|  1.0| 55.0|-32.0| 44.0| 30.52|1.634132|35.81|   S_2|
|307.25|  1.0| 49.0|-21.0| 30.0| 52.54|1.614912|35.81|   S_2|
| 307.5|  1.0| 52.0|-22.0| 27.0| 94.31|1.591848|35.81|   S_2|
|307.75|  1.0| 58.0|-14.0| 28.0|-38.86|1.558534|35.81|   S_2|
+------+-----+-----+-----+-----+------+--------+-----+------+
only showing top 5 rows

## Registering data to HDFS from an EMR to S3

In [11]:
wrists.write.option('path', '/wristdat').saveAsTable('wristdat')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Table `wristdat` already exists.;'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/readwriter.py", line 777, in saveAsTable
    self._jwrite.saveAsTable(name)
  File "/usr/lib/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.AnalysisException: 'Table `wristdat` already exists.;'



## Reading the HDFS back from S3

In [12]:
sc.stop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1579143173633_0006,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-4>

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
ss = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
ss.sql("select * from parquet.`hdfs:///wristdat`").count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

132608